In [1]:
#from prefect import flow, task
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime
from pathlib import Path
from datetime import date, timedelta
import pyarrow.parquet as pq

server = '10.40.16.19'
database = 'MSIDo'
username = 'login_fadhil'
password = 'RTvT9XBtcbjO'
TrustServerCertificate = 'yes'
engine = create_engine(f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=/opt/microsoft/msodbcsql18/lib64/libmsodbcsql-18.3.so.2.1&TrustServerCertificate={TrustServerCertificate}")
today = date.today()
yesterday = today + timedelta(-1)

def fetching_data(yesterday: str, yesterday_7: str) -> pd.DataFrame:
    server = '10.40.16.19'
    database = 'MSIDo'
    username = 'login_fadhil'
    password = 'RTvT9XBtcbjO'
    TrustServerCertificate = 'yes'

    engine = create_engine(
        f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=/opt/microsoft/msodbcsql18/lib64/libmsodbcsql-18.3.so.2.1&TrustServerCertificate={TrustServerCertificate}")
    query = (f"""
        SELECT
    u.id AS user_id,
    u.bosnet_id,
    u.phone,
    CONCAT(up.first_name, ' ', up.last_name) AS name,
    up.address,
    pc.kode_pos,
    pc.kelurahan,
    pc.kecamatan,
    r.name AS kabupaten,
    p.name AS provinsi,
    TRIM(UPPER(u.referral_by)) AS referral_by,
    CAST(up.location.STX AS NVARCHAR(MAX)) + ', ' + CAST(up.location.STY AS NVARCHAR(MAX)) AS longlat,
    FORMAT(u.created_at AT TIME ZONE 'UTC' AT TIME ZONE 'SE Asia Standard Time', 'yyyy-MM-dd HH:mm:ss') AS tanggal_regist 
    FROM users u
    INNER JOIN users_profiles up ON u.id = up.user_id
    LEFT JOIN postal_codes pc ON pc.id = up.postal_code_id 
    LEFT JOIN regencies r ON r.id = pc.regency_id 
    LEFT JOIN provinces p ON p.id = pc.province_id 

    WHERE  FORMAT(u.created_at AT TIME ZONE 'UTC' AT TIME ZONE 'SE Asia Standard Time','yyyy-MM-dd') BETWEEN '2024-01-01' AND '2024-04-02'
    ORDER BY u.created_at ASC;""")
    df = pd.read_sql(query, engine)
    df['tanggal_regist'] = pd.to_datetime(df['tanggal_regist']).dt.date
    print(f"fetching data :\n {df}")
    print(f"fetching data :\n {df.dtypes}")
    return df

def write_parquet(df: pd.DataFrame, dataset_file: str) -> Path:
    path = Path(f"/home/kantor/ingest/replicate/barkul_report/data_parquet/{dataset_file}.parquet")
    df.to_parquet(path, compression="gzip")
    return path

pd.io.parquet.get_engine('auto')

def ingest_user():
    """get data """
    today = date.today()

    yesterday = today + timedelta(-1)
#yesterday_7 = yesterday + timedelta(-6)
    df = fetching_data(str(yesterday),engine)

    """write to parquet"""
    dataset_file = f"user_{yesterday}"
    write_parquet(df, dataset_file)
    
ingest_user()


fetching data :
             user_id          bosnet_id           phone                  name  \
0     EX01202400001  0002-24-000023827   +628815432898      BK toko karunia    
1     EX01202400002  0002-24-000023829  +6288295527077      BK warung nando    
2     EX01202400003               None  +6283827862769                Aksak    
3     EX01202400004  0002-24-000023833   +628811932461  BK warung pakde nur    
4     EX01202400005               None  +6282196725783              Sanchez    
...             ...                ...             ...                   ...   
8751  EX04202400112               None  +6281252550822           Bk lumintu    
8752  EX04202400113               None  +6281357082389        Bk bunul rejo    
8753  EX04202400114               None  +6281563261900            Kamaludin    
8754  EX04202400115  0096-23-000010410  +6283131078949          ARIS JAYA 9    
8755  EX04202400116  8024-11-000000313  +6287700770779     KOPERASI AISYIAH    

                      